# REAL TIME FACE RECOGNITION 

In [1]:
# data collection and then face recognition
# K means algorithm
# Haar cascade classifier- detects each and every feature of our face
# reads the face data and stores it in an array
# easier to work with grayscale images in a knn algorithm
# KNN implements only on a numpy array

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    
    if ret==False:
        continue
    cv2.imshow('Video Frame',frame)
    
    pressed_key=cv2.waitKey(1) & 0xFF
    if pressed_key==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [3]:
#We will be passing this data in a knn algorithm
#data reading and data collection
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_alt.xml")

skip=0
face_data=[]#storing each face data
dataset_path="./face_dataset/"

file_name=input("Enter the name of person: ")

while True:
    ret,frame=cap.read()
    
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    if ret==False:
        continue
    
    faces=face_cascade.detectMultiScale(gray_frame,1.3,5)#scale, no of neighbours
    #returns the coordinates of the face captured in the form of a list(top left,top right,bottom left,bottom right)
    if len(faces)==0:
        continue
    
    k=1
    
    faces=sorted(faces,key=lambda x:x[2]*x[3],reverse=True)#Area is calculated=w*h
    #sorting in descending order on the basis of area of faces  
    
    skip+=1 #increment after each second
    
    for face in faces[:1]:
        x,y,w,h=face
        
        offset=10 #offset means padding the face detected
        #for showing a clear user interface
        face_offset=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_selection=cv2.resize(face_offset,(100,100))
        
        if skip%10==0: #10 seconds of camera
            face_data.append(face_selection)
            print(len(face_data))
            
        cv2.imshow(str(k),face_selection)
        k+=1
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
    cv2.imshow('faces',frame)
    
    key_pressed=cv2.waitKey(1) & 0xFF
    if key_pressed==ord('q'):
        break
        
face_data=np.array(face_data)
face_data=face_data.reshape((face_data.shape[0],-1))
print(face_data.shape)
            
np.save(dataset_path+file_name,face_data)
print('Dataset saved at : {}'.format(dataset_path+file_name+'.npy'))

cap.release()
cv2.destroyAllWindows()

Enter the name of person: nimish
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
(31, 30000)
Dataset saved at : ./face_dataset/nimish.npy


In [5]:
#Dataset preparation

In [4]:
def distance(v1,v2):
    #Euclidian
    return np.sqrt(((v1-v2)**2).sum())

def knn(train,test,k=5):
    dist=[]
    
    for i in range(train.shape[0]):
        ix=train[i,:-1]
        iy=train[i,-1]
        #compute the distance from test point
        d=distance(test,ix)
        dist.append([d,iy])
    dk=sorted(dist,key=lambda x:x[0])[:k]
    labels=np.array(dk)[:,-1]
    #get frequencies of each label
    output=np.unique(labels,return_counts=True)
    #find max frequency and corresponding labels
    index=np.argmax(output[1])
    return output[0][index]

cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_alt.xml")

dataset_path="./face_dataset/"

face_data=[]
labels=[]
class_id=0 #labels for every given file
names={} #mapping between id and names

#Dataset preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]#-4 means everything except last 4 elements
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        target=class_id*np.ones((data_item.shape[0],))#size of arrays will be different due to different no. of faces
        class_id+=1
        labels.append(target)
        
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)

trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)

font=cv2.FONT_HERSHEY_SIMPLEX

while True:
    ret,frame=cap.read()
    if ret==False:
        continue
        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces=face_cascade.detectMultiScale(gray,1.3,5)
    
    for face in faces:
        x,y,w,h=face
        #get the face roi
        offset=5
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        out=knn(trainset,face_section.flatten())
        
        cv2.putText(frame,names[int(out)],(x,y-10),font,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,255),2)
        
    cv2.imshow("Faces",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

(31, 30000)
(31, 1)
(31, 30001)


In [5]:
from collections import counter

def euclidian_distance(x1,x2):
    return np.sqrt(((x1-x2)**2).sum())

class KNN:
    def __init__(self,k=3):#k is the no. of nearest neighbours
        self.k=k
    
    def fit(self,X,Y):
        self.X_train=X
        self.Y_train=Y
    
    def predict(self,X):
        predicted_labels=[self._predict(x) for x in X]
        return np.array(predicted_labels)
        
    def _predict(self,x):
        #compute distances
        distances=[euclidian_distance(x,x_train) for x_train in self.X_train]
        #get k nearest samples
        k_indices=np.argsort(distances)[:self.k]
        k_nearest_labels=[self.y_train[i] for i in k_indices]
        #majority vote,most common class label
        most_common=counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

ImportError: cannot import name 'counter' from 'collections' (E:\BITS PILANI - PILANI 2021\BITS PILANI SEM 1\Anaconda (python)\lib\collections\__init__.py)